In [1]:
!pip install tensorflow-datasets


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_datasets as tfds

In [4]:
def preprocess(example):
    question = example["question"]
    passage = example["passage"]
    label = tf.cast(example["answer"], tf.float32)
    return {"question": question, "passage": passage}, label

In [16]:
train_ds = tfds.load("bool_q", split="train").map(preprocess).batch(16).prefetch(tf.data.AUTOTUNE)
val_ds = tfds.load("bool_q", split="validation").map(preprocess).batch(16).prefetch(tf.data.AUTOTUNE)

In [17]:
class USELayer(tf.keras.layers.Layer):
    def __init__(self, model_url, **kwargs):
        super().__init__(**kwargs)
        self.use = hub.load(model_url)
    def call(self, inputs):
        embeddings = self.use(inputs)
        return tf.stop_gradient(embeddings)  # freeze weights

# ---- Define inputs ----
q_in = tf.keras.Input(shape=(), dtype=tf.string, name="question")
p_in = tf.keras.Input(shape=(), dtype=tf.string, name="passage")

# ---- Apply wrapped USE ----
use_layer = USELayer("https://tfhub.dev/google/universal-sentence-encoder/4")

In [18]:
q_embed = use_layer(q_in)
p_embed = use_layer(p_in)

In [19]:
combined = tf.keras.layers.Concatenate()([
    q_embed,
    p_embed,
    tf.keras.layers.Lambda(lambda x: tf.abs(x[0] - x[1]))([q_embed, p_embed])
])

In [44]:
x = tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(1e-3))(combined)
x = tf.keras.layers.LeakyReLU(alpha=0.01)(x)
x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(1e-3))(x)
x = tf.keras.layers.LeakyReLU(alpha=0.01)(x)
x = tf.keras.layers.Dropout(0.4)(x)

out = tf.keras.layers.Dense(1, activation="sigmoid")(x)


model = tf.keras.Model(inputs=[q_in, p_in], outputs=out)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ question            │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ passage             │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ use_layer_1         │ (None, 512)       │          0 │ question[0][0],   │
│ (USELayer)          │                   │            │ passage[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 512)       │          0 │ use_layer_1[0][0… │
│                     │                   │            │ use_layer_1[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 1536)      │          0 │ use_layer_1[0][0… │
│ (Concatenate)       │                   │            │ use_layer_1[1][0… │
│                     │                   │            │ lambda_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_26 (Dense)    │ (None, 128)       │    196,736 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_15      │ (None, 128)       │          0 │ dense_26[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_18          │ (None, 128)       │          0 │ leaky_re_lu_15[0… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_27 (Dense)    │ (None, 64)        │      8,256 │ dropout_18[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_16      │ (None, 64)        │          0 │ dense_27[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_19          │ (None, 64)        │          0 │ leaky_re_lu_16[0… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_28 (Dense)    │ (None, 1)         │         65 │ dropout_19[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 205,057 (801.00 KB)

 Trainable params: 205,057 (801.00 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',   # metric to monitor
    patience=5,           # number of epochs with no improvement before stopping
    restore_best_weights=True,  # restore model weights from the epoch with best value of monitored metric
)

In [45]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,            # large number; early stopping will stop earlier
    callbacks=[early_stop]
)

Epoch 1/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 18s 50ms/step - accuracy: 0.6175 - loss: 0.9692 - val_accuracy: 0.6217 - val_loss: 0.9033
Epoch 2/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 14s 47ms/step - accuracy: 0.6301 - loss: 0.8820 - val_accuracy: 0.6217 - val_loss: 0.8417
Epoch 3/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - accuracy: 0.6301 - loss: 0.8226 - val_accuracy: 0.6226 - val_loss: 0.7965
Epoch 4/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 22s 51ms/step - accuracy: 0.6299 - loss: 0.7784 - val_accuracy: 0.6355 - val_loss: 0.7632
Epoch 5/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 16s 54ms/step - accuracy: 0.6387 - loss: 0.7445 - val_accuracy: 0.6477 - val_loss: 0.7402
Epoch 6/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 16s 54ms/step - accuracy: 0.6557 - loss: 0.7176 - val_accuracy: 0.6401 - val_loss: 0.7245
Epoch 7/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 16s 54ms/step - accuracy: 0.6658 - loss: 0.6986 - val_accuracy: 0.6462 - val_loss: 0.7112
Epoch 8/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 14s 47ms/step - accuracy: 0.6680 - loss: 0.6877 - 

In [49]:
sample = {
    "question": tf.constant(["How are you??"]),
    "passage": tf.constant(["I am good"])
}

pred = model.predict(sample)
score = float(pred[0][0])  # explicitly extract scalar
print("Prediction:", score)
print("Answer:", "True" if score > 0.5 else "False")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Prediction: 0.5580454468727112
Answer: True
